In [1]:
import pandas as pd
import os
import sys
from tqdm import tqdm
import pytz
from datetime import datetime
import json

### Resetear la configuracion de las columnas

In [2]:
pd.reset_option('display.max_rows', None)
pd.reset_option('display.max_columns', None)


### Mostrar todas las columnas

In [61]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### Mostrar todas las filas

In [38]:
pd.set_option('display.max_rows', None)

## Leer solo un csv

In [27]:
df = pd.read_csv(r'C:\Users\Desktop\Forensic\Empresas\Office365\microsoft365prueba\03f774ab-8d89-4f71-bdd0-4e2e0a53f23f.csv')


## Juntar varios csv de una carpeta en un csv solo y CAMBIAR LA FECHA A UTC-5

In [33]:
# Folders que contienen los archivos CSV

folder_path = r'C:\Users\Desktop\Forensic\Empresas\Office365\microsoft365'
output_path = r'C:\Users\Desktop\Forensic\Empresas\Office365\microsoft_junto.csv'

csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

dataframes = []

# Funcion para convertir de UTC0 a UTC-5 (Esto es editable y se puede modificar)
def convert_utc0_to_utc_minus5(date_str):
    date_str = date_str.strip()  
    if '.' in date_str:
        date_str = date_str[:date_str.index('.') + 7] + 'Z'
    utc0 = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S.%fZ')
    utc0 = utc0.replace(tzinfo=pytz.UTC)
    utc_minus5 = utc0.astimezone(pytz.timezone('Etc/GMT+5'))
    return utc_minus5.strftime('%Y-%m-%dT%H:%M:%S.%f%z')

# Leer y concatenar los archivos csv con una barra de progreso
for file in tqdm(csv_files, desc="Processing files"):
    df = pd.read_csv(os.path.join(folder_path, file))
    # Convertir de UTC0 a UTC-5
    if 'CreationDate' in df.columns:
        df['CreationDate'] = df['CreationDate'].apply(convert_utc0_to_utc_minus5)
    dataframes.append(df)

# Concatenar los Dataframes 
combined_csv = pd.concat(dataframes, ignore_index=True)

# Guardar los csv combinados en uno solo
combined_csv.to_csv(output_path, index=False)

Processing files: 100%|████████████████████████████████████████████████████████████████| 30/30 [00:23<00:00,  1.26it/s]


In [2]:
df = pd.read_csv(r'C:\Users\Desktop\Forensic\Empresas\Office365\microsoft_junto.csv')


## Agrupar, contar por Operation y User

In [ ]:
df = df.sort_values(by='CreationDate', ascending=True)
df

In [ ]:
df_operaciones = df.groupby(['UserId']).size().reset_index(name='conteo').sort_values(by='conteo', ascending=False)
df_operaciones

In [ ]:
df_operaciones = df.groupby(['Operation']).size().reset_index(name='conteo').sort_values(by='conteo', ascending=False)
df_operaciones

In [ ]:
df.groupby(['UserId','Operation']).size().reset_index(name='conteo').sort_values(by='UserId', ascending=False)

## Funcion para expandir AuditData

In [8]:
def expand_json_column(df, json_column):
    # Inicializar una lista para almacenar las filas expandidas
    expanded_rows = []

    # Iterar sobre cada fila del DataFrame con barra de progreso
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Expanding JSON column"):
        # Convertir la columna JSON a un diccionario
        json_data = json.loads(row[json_column])
        # Crear un diccionario con la fila original y los datos expandidos
        expanded_row = row.drop(labels=[json_column]).to_dict()
        expanded_row.update(json_data)
        # Agregar el diccionario a la lista de filas expandidas
        expanded_rows.append(expanded_row)

    # Crear un DataFrame a partir de las filas expandidas
    df_expanded = pd.DataFrame(expanded_rows)
    return df_expanded

In [ ]:
df_expanded_Total = expand_json_column(df, 'AuditData')
df_expanded_Total

## Revisar LoggedIn

In [ ]:
df_expanded_UserLoggedIn = df_expanded_Total.loc[df_expanded_Total['Operation'].isin(["UserLoggedIn"])].dropna(axis=1, how='all')
df_expanded_UserLoggedIn

## Leer un csv con información relevante a la IP, como pais, ciudad y ASN dentro de un nuevo dataframe 

In [78]:
df_solo_paises = pd.read_csv(r'C:\Users\Desktop\Forensic\Empresas\Office365\ips_paises.csv').drop(columns=['conteo'])


## Unir dataframe que quieres revisar con el de la informacion de la IP para hacer más filtros.

In [ ]:
df_expanded_UserLoggedIn = pd.merge(df_expanded_UserLoggedIn , df_solo_paises, on='ClientIP', how='left')
df_expanded_UserLoggedIn

# Ejemplos de tipos de consultas utiles y como es el formato de las consultas.

In [ ]:
df_expanded_UserLoggedIn.loc[(df_expanded_UserLoggedIn['Country']!='Peru') & (df_expanded_UserLoggedIn['UserId']=='userEmail')]

In [ ]:
df_expanded_UserLoggedIn.loc[(df_expanded_UserLoggedIn['Country']!='Peru') & (df_expanded_UserLoggedIn['UserId']!='userEmail')].groupby(['ClientIP','UserId','Country','City','ASN']).size().reset_index(name='conteo').sort_values(by='Country', ascending=False)

In [40]:
df_expanded_UserLoggedIn.groupby(['ClientIP','UserId']).size().reset_index(name='conteo').sort_values(by='ClientIP', ascending=False).groupby(['ClientIP']).size().reset_index(name='conteo').sort_values(by='conteo', ascending=False).to_csv("ConteoPorPaises.csv",index=False)

In [101]:
df_expanded_file.groupby(['SourceFileExtension']).size().reset_index(name='conteo').sort_values(by='conteo', ascending=False)

,SourceFileExtension,conteo
36,pdf,6417
26,jpg,5298
25,jpeg,1472
3,JPG,1329
50,xlsx,1256
37,png,346
18,docx,267
8,PNG,216
44,txt,190
47,xls,175


In [ ]:
df_expanded_file.loc[df_expanded_file["SourceFileExtension"]=="gz"]